In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''

流程解析

'''


In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''


'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("CHANNEL_ACCESS_TOKEN")
handler = WebhookHandler("CHANNEL_SECRET")

In [ ]:
'''

建置主程序的API入口

'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="text2"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="data1")
)

In [ ]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB,postbackQRB]
)

In [ ]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [ ]:
'''
訊息事件的表達
'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

specific_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")

analyze_truth_image_message=ImageSendMessage(
      original_content_url='https://i.imgur.com/M8jnn9B.png',
      preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
      )

In [ ]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          }
        ],
  )
)

In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@reply":quick_reply_text_send_message,
  "@more":buttons_template_message,
  "@123":TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號"),
  "@分析的本質":ImageSendMessage(
      original_content_url='https://i.imgur.com/M8jnn9B.png',
      preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
      )
}

In [ ]:

'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('安安，你的個資已被我紀錄了')]
    )

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    else:
        pass

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質或@more")
        )

In [ ]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''

from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dc46937283e9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"events":[{"type":"message","replyToken":"259d27b440674dea8a420296edc12d54","source":{"userId":"Ud29c5cf085e845db7c6457677116df4d","type":"user"},"timestamp":1606039859529,"mode":"active","message":{"type":"image","id":"13074981716865","contentProvider":{"type":"line"}}}],"destination":"Uf3c9a099570ca0111762abd3f35b81e4"}


127.0.0.1 - - [22/Nov/2020 10:11:03] "POST / HTTP/1.1" 200 -
